#Итоговый проект

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from os import listdir
import codecs
import pickle                                                                   
from bs4 import BeautifulSoup as BS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

In [2]:
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)       

In [11]:
path = 'content/'
texts = {}
for filename in tqdm(listdir(path)):
    doc_id = int(filename.strip('.dat'))
    if doc_id not in train_data.doc_id.values:
        continue
    with codecs.open(path + filename, 'r', 'utf-8') as f:
        url = f.readline().strip()
        html = f.read()
        bs = BS(html, 'html.parser')#ищем след страницу
        for s in bs.select('script'):
            s.extract()
        #title = bs.find_all('p')
        #title = list(map(lambda x: x.get_text(), title))
        #print(title)
        s = bs.get_text()
     #   s.translate(string.maketrans("", "", ), bad_chars)
        s=s.replace('\n',"")
        s=s.replace('\t',"")
        s=s.replace('\r',"")
        texts[doc_id] = s
        #train_data.loc[train_data.doc_id == doc_id,'text'] = s
#        train_df = train_df.append({'doc_id':int(filename.strip('.dat')),'text':s},ignore_index=True)
#        print(bs.get_text())

In [12]:
save_obj(texts,'train_data_text')
len(texts)

11648

In [82]:
train_data = pd.read_csv('train_groups.csv',dtype=np.int16)

In [6]:
texts = load_obj('train_data_text')

In [10]:
train_data['text'] = train_data.apply(lambda row: texts[row.doc_id], axis = 1)

In [83]:
train_data

,pair_id,group_id,doc_id,target
0,1,1,15731,0
1,2,1,14829,0
2,3,1,15764,0
3,4,1,17669,0
4,5,1,14852,0
...,...,...,...,...
11685,11686,129,26672,0
11686,11687,129,25838,0
11687,11688,129,25703,0
11688,11689,129,27885,0


In [8]:
train_data.shape

(11690, 5)

In [9]:
train_data[train_data.doc_id==4062]

,pair_id,group_id,doc_id,target,text
3936,3937,44,4062,1,/*** vBulletin 3.8.9 CSS* Style: 'О...


In [12]:
save_obj(train_data,'train_data')

In [ ]:
#START HEAR

In [3]:
train_data = load_obj('train_data')

In [28]:
doc_to_title = {}
with open('docs_titles.tsv', encoding = 'UTF-8') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))


28026


In [29]:
doc_titles = []
for id in train_data.doc_id.values:
    doc_titles.append(doc_to_title[id])

In [30]:
train_data['title'] = doc_titles

In [32]:
def get_splits(data,n=3):
    bound =[]
    max_group = data.group_id.max()
    bound.extend([i*int(max_group/n)+1 for i in range(1,n)])
    bound.append(max_group+1)
    prev_id = 1
    for group_id in bound:
        yield (data[~((prev_id <= data.group_id) & (data.group_id < group_id))].pair_id.values-1,
            data[(prev_id <= data.group_id) & (data.group_id < group_id)].pair_id.values-1)
        prev_id = group_id

In [54]:
vectorizer = TfidfVectorizer()#max_features = 70)
#texts_vectorized = vectorizer.fit_transform(train_data.text)

In [7]:
#texts_vectorized.shape

(11690, 1000000)

In [34]:
from sklearn.preprocessing import OneHotEncoder

In [35]:
groups_one_hot = OneHotEncoder().fit_transform(train_data.group_id.values.reshape(-1, 1))

In [36]:
groups_one_hot.shape

(11690, 129)

In [11]:
#texts_vectorized.shape

(11690, 1000000)

In [37]:
from scipy.sparse import hstack
from scipy.sparse import vstack

In [13]:
#features = hstack( [texts_vectorized,groups_one_hot] )

In [38]:
target = train_data.target.values

In [15]:
#doc_id = train_data.doc_id.values

In [85]:
''''from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
max_score = 0
for max_depth in range(5,50):
    scores = []
    for train, test in get_splits(train_data,n=3):
                print(type(features))
                model = DecisionTreeClassifier(max_depth=None)
                model.fit(features.tocsr()[train], target[train])
                pred = model.predict(features.tocsr()[test])                
                #new_pred = np.where(pred < tresh, 0, 1)
                scores.append(f1_score(pred,target[test]))
    print(sum(scores)/3)
    if sum(scores)/3 > max_score: 
        max_score = sum(scores)/3
        max_depth_best = max_depth''''
                

<class 'scipy.sparse.coo.coo_matrix'>
<class 'scipy.sparse.coo.coo_matrix'>
<class 'scipy.sparse.coo.coo_matrix'>
0.4354277501688501
<class 'scipy.sparse.coo.coo_matrix'>


KeyboardInterrupt: 

In [39]:
groups = train_data.group_id.values

In [40]:
groups

array([  1,   1,   1, ..., 129, 129, 129], dtype=int16)

In [41]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.cluster import DBSCAN

In [72]:
for group in tqdm(set(groups)):
    elements_in_same_group = train_data[train_data.group_id == group].pair_id.values-1
    texts_vectorized = vectorizer.fit_transform(train_data.title[elements_in_same_group])
    #dist_matr = pairwise_distances(texts_vectorized.tocsr()[elements_in_same_group], texts_vectorized.tocsr()[elements_in_same_group], metric = 'cosine')
    dist_matr = pairwise_distances(texts_vectorized, texts_vectorized, metric = 'cosine')
    #print(texts_vectorized.shape)
    #clustering = DBSCAN(eps = 0.5, min_samples=3).fit(dist_matr)
    dist_matr_sorted = np.sort(dist_matr, axis=1)[:,1:21]
    mean_dist = dist_matr.mean(axis = 1)
    dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist.reshape(-1,1)))

    #dist_matr_sorted = hstack((dist_matr_sorted, texts_vectorized))
    
    if group == 1 :
        features_by_dist =  dist_matr_sorted
    else:
        features_by_dist = vstack((features_by_dist, dist_matr_sorted))

In [73]:
features_by_dist.shape

(11690, 21)

In [74]:
features_by_dist = hstack([features_by_dist, groups_one_hot])

In [75]:
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
max_score = 0
for max_depth in range(5,50):
    scores = []
    for train, test in get_splits(train_data,n=3):
                print(type(features_by_dist))
                model = DecisionTreeClassifier(max_depth=None)
                model.fit(features_by_dist.tocsr()[train], target[train])
                pred = model.predict(features_by_dist.tocsr()[test])                
                scores.append(f1_score(pred,target[test]))
    print(sum(scores)/3)
    if sum(scores)/3 > max_score: 
        max_score = sum(scores)/3
        max_depth_best = max_depth

<class 'scipy.sparse.coo.coo_matrix'>
<class 'scipy.sparse.coo.coo_matrix'>
<class 'scipy.sparse.coo.coo_matrix'>
0.6480562080192915
<class 'scipy.sparse.coo.coo_matrix'>
<class 'scipy.sparse.coo.coo_matrix'>
<class 'scipy.sparse.coo.coo_matrix'>
0.6541167699349183
<class 'scipy.sparse.coo.coo_matrix'>
<class 'scipy.sparse.coo.coo_matrix'>
<class 'scipy.sparse.coo.coo_matrix'>
0.6512303658598269
<class 'scipy.sparse.coo.coo_matrix'>
<class 'scipy.sparse.coo.coo_matrix'>


KeyboardInterrupt: 